<a href="https://colab.research.google.com/github/ndkhoa0704/Spark-MLlib-project/blob/main/Lab3_Requirement1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installation
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#unzip
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

!pip install -q findspark 
!pip install pyspark

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 43.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=60b81fa3c09162cc8810187a414756d596162ac21ac202b00bc73f2ecbdf9a8c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.types as types
import pyspark.sql.functions as f
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

spark = SparkSession.builder \
    .master("local") \
    .appName("MLClassification") \
    .getOrCreate()

In [ ]:
df = spark.read.load(path='./sales_train.csv', format='csv', header=True)

In [ ]:
df.summary().show()

+-------+----------+------------------+------------------+------------------+------------------+------------------+
|summary|      date|    date_block_num|           shop_id|           item_id|        item_price|      item_cnt_day|
+-------+----------+------------------+------------------+------------------+------------------+------------------+
|  count|    433007|            433007|            433007|            433007|            433007|            433007|
|   mean|      null| 1.416554466786911| 31.06207059008284|10132.482278577483| 690.1704418760447|1.1699048745170402|
| stddev|      null|1.0866857137854327|16.484756920519086| 6299.016624157135|1127.4827786801657|0.8870135398719058|
|    min|01.01.2013|                 0|                 0|               100|              10.0|              -1.0|
|    25%|      null|               0.0|              19.0|            4464.0|             200.0|               1.0|
|    50%|      null|               1.0|              29.0|            93

In [ ]:
df.show(10)

+----------+--------------+-------+-------+----------+------------+
|      date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
|02.01.2013|             0|     59|  22154|     999.0|         1.0|
|03.01.2013|             0|     25|   2552|     899.0|         1.0|
|05.01.2013|             0|     25|   2552|     899.0|        -1.0|
|06.01.2013|             0|     25|   2554|   1709.05|         1.0|
|15.01.2013|             0|     25|   2555|    1099.0|         1.0|
|10.01.2013|             0|     25|   2564|     349.0|         1.0|
|02.01.2013|             0|     25|   2565|     549.0|         1.0|
|04.01.2013|             0|     25|   2572|     239.0|         1.0|
|11.01.2013|             0|     25|   2572|     299.0|         1.0|
|03.01.2013|             0|     25|   2573|     299.0|         3.0|
+----------+--------------+-------+-------+----------+------------+
only showing top 10 rows



In [ ]:
# Preprocess
def preprocess(df):
    t = df.withColumn('shop_id', f.col('shop_id').cast(types.DoubleType())) \
    .withColumn('item_id', f.col('item_id').cast(types.DoubleType())) \
    .withColumn('item_price', f.col('item_price').cast(types.DoubleType())) \
    .withColumn('item_cnt_day', f.col('item_cnt_day').cast(types.DoubleType()))

    # Get only rows with latest date
    t2 = t.groupBy(f.col('shop_id').alias('shop_id_t'),f.col('item_id').alias('item_id_t')).agg(f.max('date').alias('date_t'))
    cond = [t['shop_id'] == t2['shop_id_t'], t['item_id'] == t2['item_id_t'], t['date'] == t2['date_t']]
    t = t.join(t2, cond).select('shop_id', 'item_id', 'item_price', 'item_cnt_day')

    # Remove some unuseful rows
    t = t.filter(f.col('item_cnt_day') >= 0).filter(f.col('item_price') >= 0)
    return t

df = preprocess(df)

In [ ]:
# Vectorize
vecAssembler = VectorAssembler(inputCols=['item_id', 'shop_id', 'item_price'], outputCol="features")
df = vecAssembler.transform(df)

In [ ]:
# Split dataset
train_set, test_set = df.randomSplit([0.7, 0.3], seed=2022)

In [ ]:
# Regression evaluation function
def RegEval(model, test_set):
  res = model.transform(test_set)
  evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='item_cnt_day', metricName='rmse')
  print('RMSE: ', evaluator.evaluate(res))
  evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='item_cnt_day', metricName='r2')
  print('R^2: ', evaluator.evaluate(res))

# Model 1: Linear regression

In [ ]:
# Prepare model
linear_model = LinearRegression(featuresCol='features', labelCol='item_cnt_day', regParam=0.05, elasticNetParam=0.1)

# evaluator = RegressionEvaluator(labelCol="item_cnt_day",
#                                 predictionCol="prediction",
#                                 metricName="rmse")

# paramGrid = ParamGridBuilder().addGrid(linear_model.regParam, [0.05, 1, 0.2, 0.3]) \
#                               .addGrid(linear_model.elasticNetParam, [0.1, 0.2, 0.3, 0.5]).build()

# cv = CrossValidator(estimator=linear_model, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [ ]:
# Train
# cvmodel = cv.fit(train_set)
best_model_ln = linear_model.fit(train_set)

In [ ]:
# Evaluate
# best_model_ln = cvmodel.bestModel
RegEval(best_model_ln, test_set)

RMSE:  0.30646794449774745
R^2:  -1.6995792141205612e-05


# Model 2: Random Forest Regression

In [ ]:
# Prepare model
rfr = RandomForestRegressor(featuresCol='features', labelCol='item_cnt_day', numTrees=30)

# evaluator = RegressionEvaluator(labelCol="item_cnt_day",
#                                 predictionCol="prediction",
#                                 metricName="rmse")

# paramGrid = ParamGridBuilder().addGrid(rfr.numTrees, [10, 20, 30, 40]).build()

# cv = CrossValidator(estimator=rfr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [ ]:
# Train
# cvmodel = cv.fit(train_set)
best_model_rfr = rfr.fit(train_set)

In [ ]:
# Evaluate
# best_model_rfr = cvmodel.bestModel
RegEval(best_model_rfr, test_set)

RMSE:  0.3051755890811683
R^2:  0.008399234802931144


# Model 3: Gradient Boosted Regression

In [ ]:
# Prepare model
gbt = GBTRegressor(featuresCol='features', labelCol='item_cnt_day', stepSize=0.1)
# evaluator = RegressionEvaluator(labelCol="item_cnt_day",
#                                 predictionCol="prediction",
#                                 metricName="rmse")

# paramGrid = ParamGridBuilder().addGrid(gbt.maxDepth, stepSize=[0.1, 0.2, 0.5]).build()

# cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [ ]:
# Train
# cvmodel = cv.fit(train_set)
best_model_gbt = gbt.fit(train_set)

In [ ]:
# Evaluate
# best_model_gbt = cvmodel.bestModel
RegEval(best_model_gbt, test_set)

RMSE:  0.304075477265893
R^2:  0.015535490408298047
